# 데이터 집계와 그룹 연산
### 데이터 집계, 데이터 변형, 피벗 테이블: 분리->적용->결합
- groupby + (agg, apply, mean, sum, transform...)
- pivot_table

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## groupby 기본 예제

In [2]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1],
                                      dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [3]:
grouped = df.groupby(df["key1"])    # a, b, None으로 이루어진 Series
grouped

In [4]:
# 각 열에 대한 평균
grouped.mean()

,key2,data1,data2
key1,,,
a,1.5,0.555881,0.441920
b,1.5,0.705025,-0.144516


In [5]:
# key1: a, b, key2: 1, 2 -> 총 4개의 그룹 생성
means = df.groupby([df["key1"], df["key2"]]).mean()
means

data1     data2
key1 key2                    
a    1    -0.204708  0.281746
     2     0.478943  0.769023
b    1     1.965781 -1.296221
     2    -0.555730  1.007189

In [6]:
means = df.groupby(["key1", "key2"]).mean()
means

data1     data2
key1 key2                    
a    1    -0.204708  0.281746
     2     0.478943  0.769023
b    1     1.965781 -1.296221
     2    -0.555730  1.007189

### 다른 예시

In [7]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]
df[['data1','data2']].groupby([states, years]).mean()

data1     data2
CA 2005  0.936175  0.522007
   2006 -0.519439  1.246435
OH 2005 -0.380219  0.644468
   2006  1.029344 -0.533654

In [8]:
# 숫자로 이루어진 feature의 평균
df.groupby("key2").mean(numeric_only=True)

,data1,data2
key2,,
1,0.333636,0.115218
2,-0.038393,0.888106


### size: 각 feature 별로 몇 개의 샘플이 있는지 확인

In [9]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [10]:
# None인 샘플까지 그룹으로 생성 가능
df.groupby("key1", dropna=False).size()

key1
a      3
b      2
NaN    2
dtype: int64

### count: 각 그룹별로 서로 다른 값이 몇 개가 있는지 확인

In [11]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


## 그룹 간 순회하기
### 그룹 이름과 해당 그룹으로 구성된 데이터프레임 반환

In [12]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -0.204708  0.281746
1    a     2  0.478943  0.769023
5    a  <NA>  1.393406  0.274992
b
  key1  key2     data1     data2
3    b     2 -0.555730  1.007189
4    b     1  1.965781 -1.296221


In [13]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print(k1, k2)
    print(group)

a 1
  key1  key2     data1     data2
0    a     1 -0.204708  0.281746
a 2
  key1  key2     data1     data2
1    a     2  0.478943  0.769023
b 1
  key1  key2     data1     data2
4    b     1  1.965781 -1.296221
b 2
  key1  key2    data1     data2
3    b     2 -0.55573  1.007189


### 딕셔너리를 만들어서 활용 가능

In [14]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces

{'a':   key1  key2     data1     data2
 0    a     1 -0.204708  0.281746
 1    a     2  0.478943  0.769023
 5    a  <NA>  1.393406  0.274992,
 'b':   key1  key2     data1     data2
 3    b     2 -0.555730  1.007189
 4    b     1  1.965781 -1.296221}

### 열 그룹화

In [15]:
# axis 옵션을 이용해 열을 그룹화
grouped = df.groupby(["key", "key", "data", "data"], axis=1)
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -0.204708  0.281746
1  0.478943  0.769023
2 -0.519439  1.246435
3 -0.555730  1.007189
4  1.965781 -1.296221
5  1.393406  0.274992
6  0.092908  0.228913
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


/var/folders/z7/w77zh23x0dn93gjbkp2c_b500000gn/T/ipykernel_10200/3678645236.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby(["key", "key", "data", "data"], axis=1)


## 열의 일부 선택하기

In [16]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -0.204708  0.281746
     2     0.478943  0.769023
b    1     1.965781 -1.296221
     2    -0.555730  1.007189

In [17]:
# groupby 객체에서 data2만 호출
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.281746
     2     0.769023
b    1    -1.296221
     2     1.007189

In [18]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped   # 연산되지 않은 groupby 객체

In [19]:
s_grouped.mean()

key1  key2
a     1       0.281746
      2       0.769023
b     1      -1.296221
      2       1.007189
Name: data2, dtype: float64

## 다양한 방식으로 그룹화

In [20]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,1.352917,0.886429,-2.001637,-0.371843,1.669025
Steve,-0.438570,-0.539741,0.476985,3.248944,-1.021228
Wanda,-0.577087,NaN,NaN,0.523772,0.000940
Jill,1.343810,-0.713544,-0.831154,-2.370232,-1.860761
Trey,-0.860757,0.560145,-1.265934,0.119827,-1.063512


### 딕셔너리를 이용한 그룹화

In [21]:
# 각각의 feature 이름에 적용
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}

In [22]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

/var/folders/z7/w77zh23x0dn93gjbkp2c_b500000gn/T/ipykernel_10200/2922953290.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis=1)


,blue,red
Joe,-2.373480,3.908371
Steve,3.725929,-1.999539
Wanda,0.523772,-0.576147
Jill,-3.201385,-1.230495
Trey,-1.146107,-1.364125


### Series를 이용한 그룹화

In [23]:
map_series = pd.Series(mapping)
people.groupby(map_series, axis="columns").count()

/var/folders/z7/w77zh23x0dn93gjbkp2c_b500000gn/T/ipykernel_10200/4193349098.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


### 함수를 이용한 그룹화

In [24]:
# 행 기준 그룹화
people.groupby(len).sum()  #index에 함수를 적용

,a,b,c,d,e
3,1.352917,0.886429,-2.001637,-0.371843,1.669025
4,0.483052,-0.153399,-2.097088,-2.250405,-2.924273
5,-1.015657,-0.539741,0.476985,3.772716,-1.020287


### 함수와 Series를 섞어서 그룹화

In [25]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,1.352917,0.886429,-2.001637,-0.371843,1.669025
4,two,-0.860757,-0.713544,-1.265934,-2.370232,-1.860761
5,one,-0.577087,-0.539741,0.476985,0.523772,-1.021228


### agg의 인수로 메소드 이름 입력 가능 e.g. agg("mean")
- 인수로 메소드 이름/함수 배열 입력 가능 e.g. agg(["mean", "std", peak to peak])

## 데이터 집계

In [26]:
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [27]:
grouped = df.groupby("key1")
# data1 feature만 선택, key1["data1"]에서 가장 작은 값 두개만 뽑음
grouped["data1"].nsmallest(2)

key1   
a     0   -0.204708
      1    0.478943
b     3   -0.555730
      4    1.965781
Name: data1, dtype: float64

In [28]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.598113,0.494031
b,1,2.521511,2.303410


### 여러개의 열에 적용/열마다 다른 함수 적용 가능

In [29]:
tips = pd.read_csv("tips.csv")
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [30]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips["tip_pct"]

0      0.059447
1      0.160542
2      0.166587
3      0.139780
4      0.146808
         ...   
239    0.203927
240    0.073584
241    0.088222
242    0.098204
243    0.159744
Name: tip_pct, Length: 244, dtype: float64

In [31]:
grouped = tips.groupby(["day", "smoker"])

In [32]:
for a, b in grouped:
    print(a)
    print(b)

('Fri', 'No')
     total_bill   tip smoker  day    time  size   tip_pct
91        22.49  3.50     No  Fri  Dinner     2  0.155625
94        22.75  3.25     No  Fri  Dinner     2  0.142857
99        12.46  1.50     No  Fri  Dinner     2  0.120385
223       15.98  3.00     No  Fri   Lunch     3  0.187735
('Fri', 'Yes')
     total_bill   tip smoker  day    time  size   tip_pct
90        28.97  3.00    Yes  Fri  Dinner     2  0.103555
92         5.75  1.00    Yes  Fri  Dinner     2  0.173913
93        16.32  4.30    Yes  Fri  Dinner     2  0.263480
95        40.17  4.73    Yes  Fri  Dinner     4  0.117750
96        27.28  4.00    Yes  Fri  Dinner     2  0.146628
97        12.03  1.50    Yes  Fri  Dinner     2  0.124688
98        21.01  3.00    Yes  Fri  Dinner     2  0.142789
100       11.35  2.50    Yes  Fri  Dinner     2  0.220264
101       15.38  3.00    Yes  Fri  Dinner     2  0.195059
220       12.16  2.20    Yes  Fri   Lunch     2  0.180921
221       13.42  3.48    Yes  Fri   Lunch  

In [34]:
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [35]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

/var/folders/z7/w77zh23x0dn93gjbkp2c_b500000gn/T/ipykernel_10200/1734782830.py:1: FutureWarning: The provided callable <function std at 0x106b18b80> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  grouped_pct.agg([("average", "mean"), ("stdev", np.std)])


average     stdev
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [36]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [37]:
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"],
             "size" : "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [38]:
grouped = tips.groupby(["day", "smoker"])
grouped.mean(numeric_only=True)

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [39]:
# as_index 옵션을 이용해서 인덱스 작업을 하지 않음
grouped = tips.groupby(["day", "smoker"], as_index=False)
grouped.mean(numeric_only=True)

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## apply 메소드
- input이 DataFrame이 됨

In [40]:
# 내림차순 정렬 -> 상위 5개 추출하는 함수
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [41]:
# input과 output이 둘 다 DataFrame
tips.groupby("smoker").apply(top)

/var/folders/z7/w77zh23x0dn93gjbkp2c_b500000gn/T/ipykernel_10200/2530541573.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker").apply(top)


total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [42]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

/var/folders/z7/w77zh23x0dn93gjbkp2c_b500000gn/T/ipykernel_10200/643835440.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")


total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

## 분위 수 분석

In [44]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})

In [45]:
# 각 샘플별로 어디에 속했는지...
quartiles = pd.cut(frame["data1"], 4)

In [46]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
    )

grouped = frame.groupby(quartiles)  ## category를 groupby의 기준으로 삼을 수 있음
grouped.apply(get_stats)

/var/folders/z7/w77zh23x0dn93gjbkp2c_b500000gn/T/ipykernel_10200/2836935835.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = frame.groupby(quartiles)  ## category를 groupby의 기준으로 삼을 수 있음


min       max  count      mean
data1                                                     
(-2.956, -1.23] data1 -2.949343 -1.230179     94 -1.670517
                data2 -3.399312  1.670835     94 -0.079122
(-1.23, 0.489]  data1 -1.228918  0.488675    596 -0.322013
                data2 -2.989741  3.260383    596 -0.007194
(0.489, 2.208]  data1  0.489965  2.200997    300  1.066948
                data2 -3.745356  2.954439    300  0.076026
(2.208, 3.928]  data1  2.212303  3.927528     10  2.644253
                data2 -1.929776  1.765640     10  0.024750

In [47]:
grouped.aggregate(["min", "max", "count", "mean"])

data1                               data2                  \
                      min       max count      mean       min       max count   
data1                                                                           
(-2.956, -1.23] -2.949343 -1.230179    94 -1.670517 -3.399312  1.670835    94   
(-1.23, 0.489]  -1.228918  0.488675   596 -0.322013 -2.989741  3.260383   596   
(0.489, 2.208]   0.489965  2.200997   300  1.066948 -3.745356  2.954439   300   
(2.208, 3.928]   2.212303  3.927528    10  2.644253 -1.929776  1.765640    10   

                           
                     mean  
data1                      
(-2.956, -1.23] -0.079122  
(-1.23, 0.489]  -0.007194  
(0.489, 2.208]   0.076026  
(2.208, 3.928]   0.024750

In [48]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)

In [49]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -2.949343 -0.679400    250 -1.212204
      data2 -3.399312  2.628441    250 -0.056242
1     data1 -0.676853 -0.017007    250 -0.355581
      data2 -2.630247  3.260383    250 -0.001629
2     data1 -0.011862  0.625428    250  0.304229
      data2 -3.056990  2.458842    250 -0.016259
3     data1  0.627076  3.927528    250  1.253869
      data2 -3.745356  2.954439    250  0.119451

In [50]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)

In [52]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan

In [53]:
data

Ohio         -1.338659
New York      0.371224
Vermont            NaN
Florida      -0.592818
Oregon       -0.312449
Nevada             NaN
California    0.136001
Idaho              NaN
dtype: float64

In [54]:
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

East  Ohio         -1.338659
      New York      0.371224
      Vermont      -0.520084
      Florida      -0.592818
West  Oregon       -0.312449
      Nevada       -0.088224
      California    0.136001
      Idaho        -0.088224
dtype: float64

In [55]:
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])    # 각 group의 name 속성(여기에서는 West, East)
data.groupby(group_key).apply(fill_func)

East  Ohio         -1.338659
      New York      0.371224
      Vermont       0.500000
      Florida      -0.592818
West  Oregon       -0.312449
      Nevada       -1.000000
      California    0.136001
      Idaho        -1.000000
dtype: float64

### 랜덤 표본과 순열 - 간략하게 설명 후 넘어감

In [56]:
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [57]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

8S      8
JH     10
2S      2
3C      3
10C    10
dtype: int64

In [58]:
def get_suit(card):
    # last letter is suit
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  8C      8
   4C      4
D  2D      2
   QD     10
H  AH      1
   10H    10
S  6S      6
   5S      5
dtype: int64

## transform

In [59]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})

In [60]:
g = df.groupby('key')['value']
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [61]:
def get_mean(group):
    return group.mean()
g.transform(get_mean)

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [62]:
g.transform("mean")

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

apply에는 메소드 이름 입력해도 반영x

In [63]:
def times_two(group):
    return group * 2
g.transform(times_two)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [64]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [65]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

## 피벗 테이블

In [66]:
tips.pivot_table(index=["day", "smoker"],
                 values=["size", "tip", "tip_pct", "total_bill"]) ## 기본은 평균치를 집계함 

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [67]:
tips.pivot_table(index=["time", "day"], columns=["smoker","size"],
                 values=["tip_pct"])

tip_pct                                                    \
smoker             No                                                     
size                1         2         3         4         5         6   
time   day                                                                
Dinner Fri        NaN  0.139622       NaN       NaN       NaN       NaN   
       Sat   0.137931  0.162705  0.154661  0.150096       NaN       NaN   
       Sun        NaN  0.168859  0.152663  0.148143  0.206928  0.103799   
       Thur       NaN  0.159744       NaN       NaN       NaN       NaN   
Lunch  Fri        NaN       NaN  0.187735       NaN       NaN       NaN   
       Thur  0.181728  0.166005  0.084246  0.138919  0.121389  0.173706   

                                                               
smoker            Yes                                          
size                1         2         3         4         5  
time   day                                                     
Dinner Fri        NaN  0.171297       NaN  0.117750       NaN  
       Sat   0.325733  0.148668  0.144995  0.124515  0.106572  
       Sun        NaN  0.207893  0.152660  0.193370  0.065660  
       Thur       NaN       NaN       NaN       NaN       NaN  
Lunch  Fri   0.223776  0.181969       NaN       NaN       NaN  
       Thur       NaN  0.158843  0.204952  0.155410       NaN

### 그룹화를 원하는 행, 열의 feature를 결정하여 원하는 집계 방법에 따라 데이터 요약
- default는 평균치
- index: 행
- columns: 열
- values: 표시할 feature
- aggfunc: 집계 메소드

In [68]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc="count", margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244